In [1]:
!pip install pywt

ERROR: Could not find a version that satisfies the requirement pywt (from versions: none)
ERROR: No matching distribution found for pywt


In [2]:
# Import necessary libraries from TensorFlow and Keras
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2
from skimage import feature

2024-08-16 12:43:47.368947: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 12:43:47.369095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 12:43:47.500617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# Setting up file paths for training
path_train = '/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train'
path_train_img = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train/images'))
path_train_target = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/train/labels'))

# Setting up file paths for validation
path_valid = '/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid'
path_valid_img = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/images'))
path_valid_target = sorted(os.listdir('/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/labels'))

# Loading and preprocessing images and labels for training
train_images = []
train_labels = []

print("Processing training image and label files...")
for img_path, lbl_path in zip(path_train_img, path_train_target):
    img_full_path = os.path.join(path_train, 'images', img_path)
    lbl_full_path = os.path.join(path_train, 'labels', lbl_path)

    # Load the image and resize it to 224x224 pixels
    img = tf.keras.preprocessing.image.load_img(img_full_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Load the label file and parse the contents
    with open(lbl_full_path, 'r') as file:
        lbl_data = file.read().strip().split()

    if len(lbl_data) == 0:
        print(f"Warning: Label file {lbl_full_path} is empty or not formatted correctly.")
        continue  # Skip this label if it's empty

    if (len(lbl_data) - 1) % 2 != 0:
        print(f"Warning: The number of bounding box coordinates in {lbl_full_path} is not even, skipping.")
        continue  # Skip this label if the data is inconsistent

    try:
        class_label = int(lbl_data[0])  # The first value is the class label
        bounding_boxes = np.array(lbl_data[1:], dtype=np.float32).reshape(-1, 2)  # The rest are bounding box coordinates
    except Exception as e:
        print(f"Error processing label data: {e}")
        continue  # Skip this entry if there's an error

    train_images.append(img)
    train_labels.append((class_label, bounding_boxes))

# Convert lists to numpy arrays
X_train = np.array(train_images)
y_train = np.array(train_labels, dtype=object)  # Use dtype=object to store tuples

# Checkpoint: Ensure training data loading worked correctly
print(f"Loaded {X_train.shape[0]} training images with shape {X_train.shape[1:]} and {len(y_train)} training labels.")


Processing training image and label files...
Loaded 1538 training images with shape (224, 224, 3) and 1538 training labels.


In [5]:

# Loading and preprocessing images and labels for validation
valid_images = []
valid_labels = []

print("Processing validation image and label files...")
for img_path, lbl_path in zip(path_valid_img, path_valid_target):
    img_full_path = os.path.join(path_valid, 'images', img_path)
    lbl_full_path = os.path.join(path_valid, 'labels', lbl_path)

    # Load the image and resize it to 224x224 pixels
    img = tf.keras.preprocessing.image.load_img(img_full_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Load the label file and parse the contents
    with open(lbl_full_path, 'r') as file:
        lbl_data = file.read().strip().split()

    if len(lbl_data) == 0:
        print(f"Warning: Label file {lbl_full_path} is empty or not formatted correctly.")
        continue  # Skip this label if it's empty

    if (len(lbl_data) - 1) % 2 != 0:
        print(f"Warning: The number of bounding box coordinates in {lbl_full_path} is not even, skipping.")
        continue  # Skip this label if the data is inconsistent

    try:
        class_label = int(lbl_data[0])  # The first value is the class label
        bounding_boxes = np.array(lbl_data[1:], dtype=np.float32).reshape(-1, 2)  # The rest are bounding box coordinates
    except Exception as e:
        print(f"Error processing label data: {e}")
        continue  # Skip this entry if there's an error

    valid_images.append(img)
    valid_labels.append((class_label, bounding_boxes))

# Convert lists to numpy arrays
X_valid = np.array(valid_images)
y_valid = np.array(valid_labels, dtype=object)  # Use dtype=object to store tuples

# Checkpoint: Ensure validation data loading worked correctly
print(f"Loaded {X_valid.shape[0]} validation images with shape {X_valid.shape[1:]} and {len(y_valid)} validation labels.")

Processing validation image and label files...
Loaded 148 validation images with shape (224, 224, 3) and 148 validation labels.


In [6]:
# Image Augmentation using Keras for training data
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Image Augmentation using Keras for validation data
datagen_valid = ImageDataGenerator(
    rescale=1./255
)

In [7]:
from sklearn.preprocessing import StandardScaler

def normalize_features(features):
    scaler = StandardScaler()
    return scaler.fit_transform(features)

def enhance_contrast(image):
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray = image
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(gray)

def extract_edges(image):
    return cv2.Canny(image, 100, 200)

def extract_hog_features(image):
    return feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), visualize=False)

def apply_feature_engineering(image):
    enhanced = enhance_contrast(image)
    edges = extract_edges(enhanced)
    hog_features = extract_hog_features(enhanced)
    return enhanced, edges, hog_features

def process_images(image_array):
    processed_images = []
    edge_images = []
    feature_vectors = []
    for img in image_array:
        img = (img * 255).astype(np.uint8)
        enhanced, edges, features = apply_feature_engineering(img)
        processed_images.append(enhanced)
        edge_images.append(edges)
        feature_vectors.append(features)
    return np.array(processed_images), np.array(edge_images), np.array(feature_vectors)

In [8]:
# Process the training and validation datasets
X_train_processed, X_train_edges, X_train_features = process_images(X_train)
X_valid_processed, X_valid_edges, X_valid_features = process_images(X_valid)

# Normalize features
X_train_features_normalized = normalize_features(X_train_features)
X_valid_features_normalized = normalize_features(X_valid_features)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Concatenate, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

num_classes = len(set(label[0] for label in y_train))

# Define input layers
input_image = Input(shape=(224, 224, 1), name='input_image')  # Changed to 1 channel
input_features = Input(shape=(26244,), name='input_features')
input_edges = Input(shape=(224, 224, 1), name='input_edges')

# Process the main image
x = Conv2D(32, (3, 3), activation='relu')(input_image)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = GlobalAveragePooling2D()(x)

# Process the edge image
edge_x = Conv2D(16, (3, 3), activation='relu')(input_edges)
edge_x = MaxPooling2D((2, 2))(edge_x)
edge_x = Conv2D(32, (3, 3), activation='relu')(edge_x)
edge_x = GlobalAveragePooling2D()(edge_x)

# Combine all features
combined = Concatenate()([x, edge_x, input_features])

# Final dense layers
x = Dense(128, activation='relu')(combined)
output = Dense(7, activation='softmax')(x)  # 6 classes as per your model summary

# Create the model
model = Model(inputs=[input_image, input_features, input_edges], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 222, 222,  │        320 │ input_image[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 111, 111,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_edges         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 222, 222,  │        160 │ input_edges[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 54, 54,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 111, 111,  │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 52, 52,    │     36,928 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 109, 109,  │      4,640 │ max_pooling2d_2[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ conv2d_2[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ conv2d_4[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_features      │ (None, 26244)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 26340)     │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
│                     │                   │            │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │  3,371,648 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 7)         │        903 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,433,095 (13.10 MB)

 Trainable params: 3,433,095 (13.10 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

batch_size = 32

In [12]:
y_train_updated = y_train[:, 0]
y_valid_updated = y_valid[:, 0]

In [13]:
X_train_processed = X_train_processed.astype('float32')
X_train_features_normalized = X_train_features_normalized.astype('float32')
X_train_edges = X_train_edges.astype('float32')
y_train_updated = y_train_updated.astype('int')

X_valid_processed = X_valid_processed.astype('float32')
X_valid_features_normalized = X_valid_features_normalized.astype('float32')
X_valid_edges = X_valid_edges.astype('float32')
y_valid_updated = y_valid_updated.astype('int')

In [15]:
history = model.fit(
    [X_train_processed, X_train_features_normalized, X_train_edges],
    y_train_updated,
    validation_data=([X_valid_processed, X_valid_features_normalized, X_valid_edges], y_valid_updated),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)


Epoch 1/100


2024-08-16 12:45:33.086643: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 3.20911, expected 2.74785
2024-08-16 12:45:33.086697: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 3.15308, expected 2.69182
2024-08-16 12:45:33.086706: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 3.24302, expected 2.78176
2024-08-16 12:45:33.086723: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 3.36096, expected 2.8997
2024-08-16 12:45:33.086732: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.66523, expected 2.20398
2024-08-16 12:45:33.086740: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 7: 2.38489, expected 1.92363
2024-08-16 12:45:33.086747: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 2.15628, expected 1.69502
2024-08-16 12:45:33.086755: E external/local_xla/xla/ser

 5/49 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.3745 - loss: 3.2908

I0000 00:00:1723812340.469281     107 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


47/49 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5058 - loss: 2.6614

2024-08-16 12:45:42.148622: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 2.94018, expected 2.26416
2024-08-16 12:45:42.148675: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 3.11997, expected 2.44395
2024-08-16 12:45:42.148684: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.94947, expected 2.27345
2024-08-16 12:45:42.148692: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.34915, expected 1.67313
2024-08-16 12:45:42.148700: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 2.31222, expected 1.63619
2024-08-16 12:45:42.148707: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 3.18496, expected 2.50893
2024-08-16 12:45:42.148725: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.49029, expected 1.81427
2024-08-16 12:45:42.148733: E external/local_xla/xla/se

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5087 - loss: 2.6277

2024-08-16 12:45:45.430468: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49287: 2.83302, expected 2.41978
2024-08-16 12:45:45.430530: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49290: 2.91271, expected 2.49947
2024-08-16 12:45:45.430539: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49291: 2.61155, expected 2.19831
2024-08-16 12:45:45.430547: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49292: 2.65319, expected 2.23995
2024-08-16 12:45:45.430555: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49293: 2.3659, expected 1.95267
2024-08-16 12:45:45.430563: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49294: 2.50655, expected 2.09331
2024-08-16 12:45:45.430570: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49295: 2.47325, expected 2.06001
2024-08-16 12:45:45.430578: 

49/49 ━━━━━━━━━━━━━━━━━━━━ 18s 152ms/step - accuracy: 0.5101 - loss: 2.6118 - val_accuracy: 0.6216 - val_loss: 1.2533 - learning_rate: 0.0010
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.8777 - loss: 0.3450 - val_accuracy: 0.6284 - val_loss: 1.6070 - learning_rate: 0.0010
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9789 - loss: 0.0757 - val_accuracy: 0.6486 - val_loss: 1.6430 - learning_rate: 0.0010
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9934 - loss: 0.0231 - val_accuracy: 0.6284 - val_loss: 1.8074 - learning_rate: 0.0010
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9999 - loss: 0.0048 - val_accuracy: 0.6757 - val_loss: 1.5961 - learning_rate: 0.0010
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9996 - loss: 0.0026 - val_accuracy: 0.6824 - val_loss: 1.6413 - learning_rate: 0.0010
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9999 - loss: 0.0015 - val_accura

In [16]:
val_loss, val_accuracy = model.evaluate([X_valid_processed, X_valid_features_normalized, X_valid_edges], y_valid_updated)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6395 - loss: 1.2734
Validation Loss: 1.2533046007156372
Validation Accuracy: 0.6216216087341309


In [17]:
# Save the model
model.save('cnn_classification_model.h5')